# Apercal tutorial 1: Calibration and imaging of a central APERTIF beam

Now we want to actually use the pipeline to calibrate an APERTIF observation. Let's start the pipeline again as we did in the last tutorial. Remember that you need your `apercal` directory in your `$PYTHONPATH`


In [ ]:
import os;
myusername = os.environ['USER']
if not ('PYTHONPATH' in os.environ and
        '/home/' + myusername + '/apercal' in os.environ['PYTHONPATH']):
      print("WARNING: your apercal directory should be in the $PYTHONPATH")

Enable Auto-completion, import the matplotlib backend for plotting and start the logger.

In [ ]:
%config IPCompleter.greedy=True
%matplotlib notebook
import libs.lib as lib
lib.setup_logger('info', logfile='/home/{}/apercal/ipython-notebooks/tutorials/logs/1.log'.format(myusername))

Then start apercal.

In [ ]:
import apercal

Load the config file for the tutorial and look at it

In [ ]:
prepare = apercal.prepare('/home/{}/apercal/ipython-notebooks/tutorials/cfg/1.cfg'.format(myusername))
prepare.show(showall=True)

As you might guess from the name of the dataset we will be working on an M51 dataset, which was observed a while ago. This observation used the APERTIF system with 135 MHz of bandwidth in single element mode. The parameter `prepare_obsmode=single_element` only copies the central element beam over and does not care about any other beams. This also means that you can use this parameter to reduce only the central element of a multi-element observation (in case you are not interested in the rest of the field) <br>
Let the pipeline now create the needed directory structure in the given directory (parameter `basedir`) and copy over the datasets from the original directories (parameters `prepare_obsdir_fluxcal/polcal/target`)

In [ ]:
prepare.go()

You should now have a subdirectory in your `basedir + 00` (the number of the beam) + `raw` (the subdirectory for the raw datasets). To execute system tasks within the pipeline you can always import any python module you want and execute functions from them. <br>
Let's look at the contents of the directory using the `listdir` function of the `os` module. <br>
You can always access the current initial parameters by using the `module name + .[parameter]` (We will need that later to fix some issues with the dataset) 

In [ ]:
import os
os.listdir(prepare.basedir + prepare.beam + '/' + prepare.rawsubdir)

We now have two datasets in this subdirectory:
 - 3C147.MS (calibrator)
 - M51.MS (target) <br>

Let's continue with the flagging step.

In [ ]:
preflag = apercal.preflag('/home/{}/apercal/ipython-notebooks/tutorials/cfg/1.cfg'.format(myusername))
preflag.show()

We will do manual flagging and after that use the aoflagger to automatically find RFI (parameters `preflag_manualflag=True` and `preflag_aoflagger=True`). Unfortunately, since we are using some CASA routines, it messes up the logger and crashes the pipeline, if we execute the whole module at once with `preflag.go()`. <br>
To circumvent this we will execute the individual substeps one by one. For the manual flagging we will flag the auto-correlations (`preflag_manualflag_auto = True`) and flag for shadowing (`preflag_manualflag_shadow = True`).

In addition we need to create a list of channels to flag the subband edges and the two channels affected by ghosts, which we then insert into the `preflag_manualflag_channel` parameter. The total number of channels is 11008.

In [ ]:
nchannel = 11008

Then we need to make a list of channels which are affected, combine them into a list and convert to a string

In [ ]:
a = range(0, nchannel, 64) # the subband edges
b = range(1, nchannel, 64)
c = range(63, nchannel, 64)
d = range(16, nchannel, 64) # the two ghosts
e = range(48, nchannel, 64)

l = a + b + c + d + e
preflag.preflag_manualflag_channel = ';'.join(str(ch) for ch in l)

preflag.show()

Now execute the `manualflag` substep.

In [ ]:
preflag.manualflag()

We want to derive a bandpass and apply it to the data before we use aoflagger (`parameter preflag_aoflagger_bandpass = True`). This bandpass will not be used in later stages of the calibration process and helps aoflagger to distinguish between bandpass effects and real RFI better. There will be an error message at the end of the step which you can ignore.

In [ ]:
preflag.aoflagger_bandpass()

Execute the aoflagger substep now.

In [ ]:
preflag.aoflagger_flag()

In [ ]:
convert = apercal.convert('/home/{}/apercal/ipython-notebooks/tutorials/cfg/1.cfg'.format(myusername))
convert.show()

The convert step coverts the datasets from MS format to UVFITS format (`convert_ms2uvfits = True`) and then to MIRIAD format (`convert_uvfits2mir = True`). We do not want to carry the auto-correlations over (`convert_ms2uvfits_tool_casa_autocorr = False`). The converted datasets are saved in the crosscal subdirectory.

In [ ]:
convert.go()

Since this is an old dataset (anything after 21.12.17 should be fine), we need to recompute the uv-coordinates. This can be done manually using the task `uvedit` in MIRIAD. We change the pointing centre by an infinitesimal amount (0.00001s) to force `uvedit` to recompute them.

Initialise the task.

In [ ]:
import os
uvedit = lib.miriad('uvedit')

Execute the task first the calibrator and the for the target.

In [ ]:
uvedit.vis = convert.crosscaldir + '/' + convert.fluxcal.rstrip('MS') + 'mir'
uvedit.ra = 0.00001
uvedit.out = convert.crosscaldir + '/' + convert.fluxcal.rstrip('.MS') + '_uvedit.mir'
uvedit.go()

uvedit.vis = convert.crosscaldir + '/' + convert.target.rstrip('MS') + 'mir'
uvedit.ra = 0.00001
uvedit.out = convert.crosscaldir + '/' + convert.target.rstrip('.MS') + '_uvedit.mir'
uvedit.go()

Remove the obsolete files from the directories

In [ ]:
os.system('rm -rf ' + convert.crosscaldir + '/' + convert.fluxcal.rstrip('MS') + 'mir')
os.system('rm -rf ' + convert.crosscaldir + '/' + convert.target.rstrip('MS') + 'mir')
os.system('mv ' + convert.crosscaldir + '/' + convert.fluxcal.rstrip('.MS') + '_uvedit.mir ' + convert.crosscaldir + '/' + convert.fluxcal.rstrip('.MS') + '.mir')
os.system('mv ' + convert.crosscaldir + '/' + convert.target.rstrip('.MS') + '_uvedit.mir ' + convert.crosscaldir + '/' + convert.target.rstrip('.MS') + '.mir')

Since this is an old dataset (anything after 21.12.17 should be fine), we need to flip the phases. This can be done, in the same manner as before, using the task `uvrot` in MIRIAD.

In [ ]:
import os

uvrot = lib.miriad('uvrot')

uvrot.vis = convert.crosscaldir + '/' + convert.fluxcal.rstrip('.MS') + '.mir'
uvrot.angle = 180
uvrot.out = convert.crosscaldir + '/' + convert.fluxcal.rstrip('.MS') + '_uvrot.mir'
uvrot.go()

uvrot.vis = convert.crosscaldir + '/' + convert.target.rstrip('.MS') + '.mir'
uvrot.angle = 180
uvrot.out = convert.crosscaldir + '/' + convert.target.rstrip('.MS') + '_uvrot.mir'
uvrot.go()

os.system('rm -rf ' + convert.crosscaldir + '/' + convert.fluxcal.rstrip('MS') + 'mir')
os.system('rm -rf ' + convert.crosscaldir + '/' + convert.target.rstrip('MS') + 'mir')
os.system('mv ' + convert.crosscaldir + '/' + convert.fluxcal.rstrip('.MS') + '_uvrot.mir ' + convert.crosscaldir + '/' + convert.fluxcal.rstrip('.MS') + '.mir')
os.system('mv ' + convert.crosscaldir + '/' + convert.target.rstrip('.MS') + '_uvrot.mir ' + convert.crosscaldir + '/' + convert.target.rstrip('.MS') + '.mir')

In [ ]:
ccal = apercal.ccal('/home/{}/apercal/ipython-notebooks/tutorials/cfg/1.cfg'.format(myusername))
ccal.show()

Crosscal derives the gains and bandpass (`crosscal_bandpass = True`) from the calibrator and applies them to the target field (`crosscal_transfer_to_target = True`). We also want to solve for the delays. This will work better for recent datasets (since we have the cable delay compensation in place), but still might need to be done (the delay compensation can only be sampled in increments of 5ns).

In [ ]:
ccal.go()

In [ ]:
scal = apercal.scal('/home/{}/apercal/ipython-notebooks/tutorials/cfg/1.cfg'.format(myusername))
scal.show()

For the self-calibration we are using an image size of 1025 pixels (`selfcal_image_imsize = 1025`) with a cell size of 8 arcseconds (`selfcal_image_cellsize = 8`). We split the data (`selfcal_splitdata = True`) in chunks of 0.02 GHz (`selfcal_splitdata_chunkbandwidth = 0.02`) and average to a channel bandwidth of 0.001 GHz (`selfcal_splitdata_channelbandwidth = 0.001`). The algorithm will split the observed band uniformly between the chunks. The used chunk and chanel bandwidth will therefore be close but not exactly the given ones. <br>
We do not do any parametric self-calibration (`selfcal_parametric = False`) since our observed target field is dominated by diffuse emission. We use 3 majorcycles (`selfcal_standard_majorcycle = 3`) with 5 minorcycles each (`selfcal_standard_minorcycle = 5`). Both types of cycles use a square function in calculate the maximum increment in dynamic range between the cycles (`selfcal_standard_majorcycle_function = square/selfcal_standard_minorcycle_function = square`). <br>
We use the complete uv-range for every cycle (`selfcal_standard_uvmin = [0.0, 0.0, 0.0]/selfcal_standard_uvmax = [3000, 3000, 3000]`) and solution intervals of 5,3, and 0.5 minutes for the three major cycles (`selfcal_standard_solint = [5, 3, 0.5]`).
We do not use any amplitude calibration (`selfcal_standard_amp = False`). You can experiment with options `True` (always use amplitude calibration) or `'auto'` (use amplitude calibration if the clean components reach a flux of selfcal_standard_amp_auto_limit Jy). <br>
Only one solution is used for each individual chunk (`selfcal_standard_nfbin = 1`).

In [ ]:
scal.go()

All frequency chunks were calibrated and it is now time to create a final continuum image.

In [ ]:
continuum = apercal.continuum('/home/{}/apercal/ipython-notebooks/tutorials/cfg/1.cfg'.format(myusername))
continuum.show()

We are going to use the same image size and cellsize as for the self-calibration (`continuum_image_imsize = 1025/continuum_image_cellsize = 8`). Since we are stacking all the final images of the individual chunks in the end (`continuum_mode = stack`) we want to convolve with a commen beam (`continuum_image_convolbeam = 45,45,0.0`). We use the same paramaters as for the self-calibration for the final cleaning procedure.

In [ ]:
continuum.go()

Since the deep continuum images are ready now, we can use this model to subtract the continuum and image the line channels. In case of a non-existent continuum model from the continuum step, the line module would automatically image the continuum beforehand and use it for subtraction.

In [ ]:
line = apercal.line('/home/{}/apercal/ipython-notebooks/tutorials/cfg/1.cfg'.format(myusername))
line.show()

The line module is using the originally converted MIRIAD data files, splits and compresses them to the desired channel bandwidth (parameters: `line_splitdata_chunkbandwidth` and `line_splitdata_channelbandwidth`). Then it transfers the gains from the self calibration (`line_transfergains`) and applies them before subtracting the continuum model (`line_subtract = True/line_subtract_mode = uvmodel`).
Imaging of the line cube (`line_image = True`) is done with the same image size (`line_image_imsize = 1025`) and cell size (`line_image_cellsize = 8`) as for the continuum.
We use a robust weighting of -2 for the imaging (`line_image_robust = -2`) and clean channels where the ratio between the maximum and minimum pixel value is larger than 1.05 (`line_image_ratio_limit = 1.05`).
We only image the channel range between channel 9650 and 9800 (`line_image_channels = 9650,9800`) and all images are covolved to 45 arcseconds (`line_image_convolbeam = 45,45,0.0`).

In [ ]:
line.go()